In [100]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time
import requests
import csv
from selenium import webdriver
from pprint import pprint
from scraper import scrape_site
from pathlib import Path
import pandas as pd
from config import subscription_key
from urllib.error import HTTPError
from googleapi import google
from urllib.parse import urlencode, urlparse, parse_qs, urlsplit

import requests.exceptions
from collections import deque

from lxml.html import fromstring
from requests import get

In [2]:
file_to_output = Path("Data_Scraped/Contacts.csv")

In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', headless=False, **executable_path)

In [4]:
url = 'https://www.bizjournals.com/houston/potm?l=&time=&type=&ind=86'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
Final_List = []
dict_list =[]
links = []

In [13]:

for link in soup.find_all("a", class_="item", href=True):
        links.append(link.get('href'))
#Get Next button
for link in soup.find_all("a", class_="btn btn--primary pull-right", href=True):
    button = link.get('href')


for i in links:
    if '/houston/potmsearch/detail/submission' in i:
        d = {}
        link = "https://www.bizjournals.com/" + i
        newI = i.split("?")
        slicefront = (newI[0])
        name = slicefront[46:]
        final_name = name.replace('_', " ")
        d["Name"] = final_name
        d["Link"] = link
        if d not in dict_list:
            dict_list.append(d)
        else:
            pass
    else:
        pass

In [14]:

try:
#     browser.click_link_by_partial_href('Next')
    browser.click_link_by_href(button) 
#     browser.find_link_by_text('Next').first.click()
#     browser.links.find_by_partial_text("Next").click()
    print("Went to the next page")
    html = ''
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
except:
    print("Finished all pages")


Finished all pages


In [15]:
len(dict_list)

87

In [16]:
pprint(dict_list)

[{'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489460/Bethany_Spare?l=&time=&ind=86&type=&ro=0',
  'Name': 'Bethany Spare'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6488386/Charles_M_Rosson?l=&time=&ind=86&type=&ro=1',
  'Name': 'Charles M Rosson'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6487947/Michael_Hamilton?l=&time=&ind=86&type=&ro=2',
  'Name': 'Michael Hamilton'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6486774/Jonathan_Newton?l=&time=&ind=86&type=&ro=3',
  'Name': 'Jonathan Newton'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6485908/John_Farrell?l=&time=&ind=86&type=&ro=4',
  'Name': 'John Farrell'},
 {'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6485383/Connie_Pfeiffer?l=&time=&ind=86&type=&ro=5',
  'Name': 'Connie Pfeiffer'},
 {'Link': 'https://www.bizjournals.com//housto

In [17]:
for i in dict_list:

        try:
                browser.visit(i["Link"])
                t.sleep(1)
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                info_list = soup.find_all('p', class_ = 'cXenseParse')
                try: 
                    i["Title"] = info_list[0].text
                    i["Blurb"] = info_list[1].text
                except:
                    print(f"Missing info for {i['Name']}")
                Final_List.append(i)

        except HTTPError as error:
            print(error)

Missing info for Randy SorrelsPresident State Bar of Texas


In [18]:
len(Final_List)

87

In [19]:
pprint(Final_List)

[{'Blurb': 'Bethany Spare represents clients in complex commercial litigation '
           'and appeals. Bethany graduated from The University of Texas School '
           'of Law where she served as Submissions Editor for Texas Review of '
           'Law & Politics. Prior to her arrival, Bethany was a Gregory S. '
           "Coleman Fellow with the Texas Attorney General's Office and "
           'clerked with the Hon. James Ho of the U.S. Court of Appeals for '
           'the Fifth Circuit and the Hon. Reed O’Connor for the U.S. District '
           'Court for the Northern District of Texas.',
  'Link': 'https://www.bizjournals.com//houston/potmsearch/detail/submission/6489460/Bethany_Spare?l=&time=&ind=86&type=&ro=0',
  'Name': 'Bethany Spare',
  'Title': 'Associate at Yetter Coleman LLP'},
 {'Blurb': 'Charles Rosson devotes his practice to complex commercial '
           'litigation disputes, representing both plaintiffs and defendants '
           'in state and federal court a

In [20]:
FL = pd.DataFrame(Final_List)
FL.head()

,Name,Link,Title,Blurb
0,Bethany Spare,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...
1,Charles M Rosson,https://www.bizjournals.com//houston/potmsearc...,Partner at Gibbs & Bruns LLP,Charles Rosson devotes his practice to complex...
2,Michael Hamilton,https://www.bizjournals.com//houston/potmsearc...,Partner at King & Spalding LLP,Michael Hamilton represents public and private...
3,Jonathan Newton,https://www.bizjournals.com//houston/potmsearc...,Partner at King & Spalding LLP,"King & Spalding has added Jonathan B. Newton, ..."
4,John Farrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Arcadi Jackson LLP,John Farrell has more than 13 years of legal e...


In [21]:
pd.set_option('display.max_rows', None)
FL

,Name,Link,Title,Blurb
0,Bethany Spare,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...
1,Charles M Rosson,https://www.bizjournals.com//houston/potmsearc...,Partner at Gibbs & Bruns LLP,Charles Rosson devotes his practice to complex...
2,Michael Hamilton,https://www.bizjournals.com//houston/potmsearc...,Partner at King & Spalding LLP,Michael Hamilton represents public and private...
3,Jonathan Newton,https://www.bizjournals.com//houston/potmsearc...,Partner at King & Spalding LLP,"King & Spalding has added Jonathan B. Newton, ..."
4,John Farrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Arcadi Jackson LLP,John Farrell has more than 13 years of legal e...
5,Connie Pfeiffer,https://www.bizjournals.com//houston/potmsearc...,Partner at Yetter Coleman LLP,Connie Pfeiffer has joined Yetter Coleman as a...
6,Kathlyn Hufstetler,https://www.bizjournals.com//houston/potmsearc...,Senior Associate Attorney at Patel | Gaines,Not only does Kathlyn litigate commercial prop...
7,Sammy Ford IV,https://www.bizjournals.com//houston/potmsearc...,Partner at AZA Law Firm,"A highly acclaimed lawyer, Mr. Ford has tried ..."
8,Colleen Migl,https://www.bizjournals.com//houston/potmsearc...,Shareholder at Kilmer Crosby & Quadros,"Colleen Migl, who will be working out of the f..."
9,Brooks Tueting,https://www.bizjournals.com//houston/potmsearc...,partner at Patterson + Sheridan LLP,Intellectual property and complex litigation l...


In [22]:
FL.isna().sum()

Name     0
Link     0
Title    0
Blurb    1
dtype: int64

In [23]:
file_to_output = Path("Data_Scraped/FinalContacts.csv")

In [24]:
# Saving the dataframe to a csv
df_to_csv = FL.to_csv(file_to_output)

## Searching Bing for website and emails

In [80]:
assert subscription_key


In [90]:
# Set the URL to bing API
search_url = "https://api.bing.microsoft.com/v7.0/search"

In [91]:
search_term = "Bethany Spare Associate at Yetter Coleman LLP"

In [92]:
headers = {"Ocp-Apim-Subscription-Key": "5158c76dee144128862504b6b30ae70a"}
params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = response.json()
search_results

{'_type': 'SearchResponse',
 'queryContext': {'originalQuery': 'Bethany Spare Associate at Yetter Coleman LLP'},
 'webPages': {'webSearchUrl': 'https://www.bing.com/search?q=Bethany+Spare+Associate+at+Yetter+Coleman+LLP',
  'totalEstimatedMatches': 313000,
  'value': [{'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0',
    'name': 'Spare, Bethany C. - Yetter Coleman LLP',
    'url': 'https://www.yettercoleman.com/profiles/spare-bethany-c/',
    'isFamilyFriendly': True,
    'displayUrl': 'https://www.<b>yettercoleman</b>.com/profiles/<b>spare</b>-<b>bethany</b>-c',
    'snippet': '<b>Bethany</b>’s practice focuses on complex commercial litigation and appeals. She graduated from The University of Texas School of Law in 2017, serving as the Submissions Editor for the Texas Review of Law &amp; Politics.. Prior to joining the firm, <b>Bethany</b> was a Gregory S. <b>Coleman</b> Fellow with the Texas Attorney General’s Office.',
    'dateLastCrawled': '2020-11-20T01:04:00.0000000Z',

In [93]:
search_results['webPages']['value'][0]['url']

'https://www.yettercoleman.com/profiles/spare-bethany-c/'

In [94]:
for i in Final_List:
    try:
        search_url = "https://api.bing.microsoft.com/v7.0/search"
        search_term = i["Name"] + ' ' + i["Title"]
        headers = {"Ocp-Apim-Subscription-Key": subscription_key}
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()
        i['Bing Search Link'] = search_results['webPages']['value'][0]['url']
        print(f"Found {i['Name']}!")
    except:
        print(f"Could not find {i['Name']}")
    time.sleep(1)

Found Bethany Spare!
Found Charles M Rosson!
Found Michael Hamilton!
Found Jonathan Newton!
Found John Farrell!
Found Connie Pfeiffer!
Found Kathlyn Hufstetler!
Found Sammy Ford IV!
Found Colleen Migl!
Found Brooks Tueting!
Found Edward Ed L Ripley!
Found Stephanie McGraw!
Found Ben Walther!
Found Michael L Telford!
Found Elizabeth Liz Webb!
Found Caroline C Pace!
Found Keith A Taylor!
Found Gabriela Gabby M Barake!
Found Stephen Jacobson!
Found Justo Mendez!
Found Samuel Louis!
Found S Priya Coffey!
Found Joe Flack!
Found Christopher Hogan!
Found Samantha Thompson!
Found David W Parker!
Found Kristhy Peguero!
Found William AI Trey McDonald III!
Found Briggs Knight!
Found Kate Kaufmann Shih!
Found Lindsey Raspino!
Found Kenton Andrews!
Found Dorothy Watson!
Found Debbie Pacholder!
Found Mike Seiler!
Found Kenna Seiler!
Found Steven Mitby!
Found C Maxwell Rizzo!
Found Cindy Muller!
Found Jim Noe!
Found Amy Anderson!
Found Loren Canales!
Found Sara Valentine!
Found Wendy Burgower!
Found 

In [96]:
FL = pd.DataFrame(Final_List)
FL.head()

,Name,Link,Title,Blurb,Bing Search Link
0,Bethany Spare,https://www.bizjournals.com//houston/potmsearc...,Associate at Yetter Coleman LLP,Bethany Spare represents clients in complex co...,https://www.yettercoleman.com/profiles/spare-b...
1,Charles M Rosson,https://www.bizjournals.com//houston/potmsearc...,Partner at Gibbs & Bruns LLP,Charles Rosson devotes his practice to complex...,https://www.gibbsbruns.com/people/charles-m-ro...
2,Michael Hamilton,https://www.bizjournals.com//houston/potmsearc...,Partner at King & Spalding LLP,Michael Hamilton represents public and private...,https://www.linkedin.com/in/michaelhamiltontx
3,Jonathan Newton,https://www.bizjournals.com//houston/potmsearc...,Partner at King & Spalding LLP,"King & Spalding has added Jonathan B. Newton, ...",https://www.linkedin.com/in/jonathanbnewton
4,John Farrell,https://www.bizjournals.com//houston/potmsearc...,Partner at Arcadi Jackson LLP,John Farrell has more than 13 years of legal e...,https://www.linkedin.com/in/john-m-farrell-b13...


In [97]:
FL.isna().sum()

Name                0
Link                0
Title               0
Blurb               1
Bing Search Link    0
dtype: int64

In [98]:
FL.Title.value_counts()

Partner at Jackson Walker LLP                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               6
Associate at Yetter Coleman LLP                                                                                                                                                                           

In [99]:
# Saving the dataframe to a csv
df_to_csv = FL.to_csv(file_to_output)

In [107]:
#Convert Bing Links to a list
BSL = FL["Bing Search Link"].to_list()
BSL

['https://www.yettercoleman.com/profiles/spare-bethany-c/',
 'https://www.gibbsbruns.com/people/charles-m-rosson/',
 'https://www.linkedin.com/in/michaelhamiltontx',
 'https://www.linkedin.com/in/jonathanbnewton',
 'https://www.linkedin.com/in/john-m-farrell-b13b42185',
 'https://www.yettercoleman.com/profiles/pfeiffer-connie-h/',
 'https://www.usatoday.com/story/entertainment/celebrities/2020/11/14/joanna-gaines-wishes-chip-gaines-happy-birthday-touching-tribute/6296723002/',
 'https://azalaw.com/our-attorneys/sammy-ford-iv/',
 'https://www.kcq-lawfirm.com/',
 'https://www.pattersonsheridan.com/brooks-tueting/',
 'https://www.andrewsmyers.com/ripleybankruptcy2020/',
 'https://www.shb.com/professionals/m/mcgraw-stephanie',
 'https://www.shb.com/professionals/w/walther-benjamin',
 'http://www.nathansommers.com/Attorneys/Michael_L_Telford/',
 'https://www.craincaton.com/attorneys/elizabeth-liz-webb/',
 'https://www.craincaton.com/attorneys/caroline-c-pace/',
 'https://www.craincaton.com/

In [101]:
# starting url. replace google with your own url.
starting_url = 'https://www.kslaw.com/people?locale=en&office_id=4'

# a queue of urls to be crawled
unprocessed_urls = deque([starting_url])

# set of already crawled urls for email
processed_urls = set()

# a set of fetched emails
emails = set()

# process urls one by one from unprocessed_url queue until queue is empty
while len(unprocessed_urls):

    # move next url from the queue to the set of processed urls
    url = unprocessed_urls.popleft()
    processed_urls.add(url)

    # extract base url to resolve relative links
    parts = urlsplit(url)
    base_url = "{0.scheme}://{0.netloc}".format(parts)
    path = url[:url.rfind('/')+1] if '/' in parts.path else url

    # get url's content
    print("Crawling URL %s" % url)
    try:
        response = requests.get(url)
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
        # ignore pages with errors and continue with next url
        continue

    # extract all email addresses and add them into the resulting set
    # You may edit the regular expression as per your requirement
    new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", response.text, re.I))
    emails.update(new_emails)
    print(emails)
    # create a beutiful soup for the html document
    soup = BeautifulSoup(response.text, 'lxml')
    # Once this document is parsed and processed, now find and process all the anchors i.e. linked urls in this document
    for anchor in soup.find_all("a"):
        # extract link url from the anchor
        link = anchor.attrs["href"] if "href" in anchor.attrs else ''
        # resolve relative links (starting with /)
        if link.startswith('/'):
            link = base_url + link
        elif not link.startswith('http'):
            link = path + link
        # add the new url to the queue if it was not in unprocessed list nor in processed list yet
        if not link in unprocessed_urls and not link in processed_urls:
            unprocessed_urls.append(link)

Crawling URL https://www.kslaw.com/people?locale=en&office_id=4
{'cburke@kslaw.com', 'kcagle@kslaw.com', 'rgarner@kslaw.com', 'raguirreluzi@kslaw.com', 'sberry@kslaw.com', 'pbessette@kslaw.com', 'mburke@kslaw.com', 'wcoriell@kslaw.com', 'ccorrell@kslaw.com', 'jjanderson@kslaw.com', 'dbishop@kslaw.com', 'bgordon@kslaw.com'}


In [69]:
for i in Final_List:
    try:
        search_url = "https://api.cognitive.microsoft.com/bing/v7.0/search"
        search_term = i["Name"] + ' ' + i["Title"]
        headers = {"Ocp-Apim-Subscription-Key": subscription_key}
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get(search_url, headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()
        i['Bing Search Link'] = search_results['webPages']['value'][0]['url']
        print(f"Found {i['Name']}!")
    except:
        print(f"Could not find {i['Name']}")
    time.sleep(1)


Could not find Bethany Spare
Could not find Charles M Rosson
Could not find Michael Hamilton
Could not find Jonathan Newton
Could not find John Farrell
Could not find Connie Pfeiffer
Could not find Kathlyn Hufstetler
Could not find Sammy Ford IV
Could not find Colleen Migl
Could not find Brooks Tueting
Could not find Edward Ed L Ripley
Could not find Stephanie McGraw
Could not find Ben Walther
Could not find Michael L Telford
Could not find Elizabeth Liz Webb
Could not find Caroline C Pace
Could not find Keith A Taylor
Could not find Gabriela Gabby M Barake
Could not find Stephen Jacobson
Could not find Justo Mendez
Could not find Samuel Louis
Could not find S Priya Coffey
Could not find Joe Flack
Could not find Christopher Hogan
Could not find Samantha Thompson
Could not find David W Parker
Could not find Kristhy Peguero
Could not find William AI Trey McDonald III
Could not find Briggs Knight


KeyboardInterrupt: 

In [35]:
search_url = "https://api.bing.microsoft.com/v7.0/search"

In [36]:
search_term = "Microsoft Bing Search Services"

In [41]:
headers = {"Ocp-Apim-Subscription-Key": subscription_key}
params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = response.json()
type(search_results)

dict

In [39]:
pprint(search_results)

{'_type': 'SearchResponse',
 'entities': {'value': [{'bingId': '1878d09e-59d3-4e8c-8580-ca67d2ec2d2d',
                         'contractualRules': [{'_type': 'ContractualRules/LicenseAttribution',
                                               'license': {'name': 'CC-BY-SA',
                                                           'url': 'http://creativecommons.org/licenses/by-sa/3.0/'},
                                               'licenseNotice': 'Text under '
                                                                'CC-BY-SA '
                                                                'license',
                                               'mustBeCloseToContent': True,
                                               'targetPropertyName': 'description'},
                                              {'_type': 'ContractualRules/LinkAttribution',
                                               'mustBeCloseToContent': True,
                                              

                        {'dateLastCrawled': '2020-11-14T04:02:00.0000000Z',
                         'displayUrl': 'https://account.<b>microsoft</b>.com/account/<b>bing</b>',
                         'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.9',
                         'isFamilyFriendly': True,
                         'isNavigational': False,
                         'language': 'en',
                         'name': 'Microsoft account | Sign In or Create Your '
                                 'Account Today ...',
                         'snippet': 'Access your favorite <b>Microsoft</b> '
                                    'products and <b>services</b> with just '
                                    'one login. From Office and Windows to '
                                    'Xbox and Skype, one username and password '
                                    'connects you to the files, photos, '
                                    'people, and content you care about most.

In [ ]:
# for i in Final_List:
#             if 'Title' in i:
#                 query = str(i['Name']) + ' ' + str(i["Title"])
#                 # Using the first page of Google
#                 page = 1
#                 start = (page - 1) * 10 + 1
#                 url = f"https://www.googleapis.com/customsearch/v1?key={API_KEY}&cx={SEARCH_ENGINE_ID}&q={query}&start={start}"
#                 data = requests.get(url).json()
#                 search_items = data.get("items")
                
#                 link = search_items[0].get("link")
#                 i["Business Link"] = link
#             elif 'Name' in i:
#                 query = str(i['Name'])
#                 page = 1
#                 start = (page - 1) * 10 + 1
#                 url = f"https://www.googleapis.com/customsearch/v1?key={API_KEY}&cx={SEARCH_ENGINE_ID}&q={query}&start={start}"
#                 data = requests.get(url).json()
#                 search_items = data.get("items")
#                 link = search_items[0].get("link")
#                 i["Business Link"] = link
#             else:
#                 print(f"Could not find {i['Name']}")
        

In [ ]:
# for i in Final_List:
#     query = str(i['Name']) + ' ' + str(i["Title"])
#     search_results = google.search(query)
#     link = self.link

In [ ]:
# query = Final_List[0]['Name'] + ' ' + Final_List[0]['Title']
# search_results = google.search(query)
# search_results[0]

In [ ]:
# search_results[0]

In [ ]:
# Final_List[0]['Name'] + ' ' + Final_List[0]['Title']

In [ ]:
# Final_List

In [ ]:
# query = Final_List[0]['Name'] + ' ' + Final_List[0]['Title']
# # Using the first page of Google
# page = 1
# start = (page - 1) * 10 + 1
# url = f"https://www.googleapis.com/customsearch/v1?key={API_KEY}&cx={SEARCH_ENGINE_ID}&q={query}&start={start}"
# data = requests.get(url).json()
# search_items = data.get("items")
# print(data)

In [ ]:
# query = Final_List[0]['Name'] + '' + Final_List[0]['Title']
# raw = get(f"https://www.google.com/search?q=query").text


# print(raw)

In [ ]:
# class="yuRUbf"

In [ ]:
# query = Final_List[0]['Name'] + ' ' + Final_List[0]['Title']
# url = f"https://www.google.com/search?q={query}"
# browser.visit(url)
# html = browser.html
# soup = BeautifulSoup(html, 'html.parser')


In [ ]:
# link =  soup.find("div", { "class" : "yuRUbf" }).find("a", recursive=False, href=True)
# the_link = link.get('href')
# the_link

In [ ]:
# for i in Final_List:
#         try:
#             if 'Title' in i:
#                 query = Final_List[0]['Name'] + ' ' + Final_List[0]['Title']
#                 # Using the first page of Googl
#                 url = f"https://www.google.com/search?q={query}"
#                 browser.visit(url)
#                 html = browser.html
#                 soup = BeautifulSoup(html, 'html.parser')
#                 link =  soup.find("div", { "class" : "yuRUbf" }).find("a", recursive=False, href=True)
#                 the_link = link.get('href')
#                 i["Business Link"] = the_link
#             elif 'Name' in i:
#                 query = Final_List[0]['Name']
#                 # Using the first page of Googl
#                 url = f"https://www.google.com/search?q={query}"
#                 browser.visit(url)
#                 html = browser.html
#                 soup = BeautifulSoup(html, 'html.parser')
#                 link =  soup.find("div", { "class" : "yuRUbf" }).find("a", recursive=False, href=True)
#                 the_link = link.get('href')
#                 i["Business Link"] = the_link
#             else:
#                 print(f"Could not find {i['Name']}")
#             sleep(1)
#         except:
#             print(f'Could not find {i["Name"]}')